In [113]:
#!pip install dgl-cu113 dglgo -f https://data.dgl.ai/wheels/repo.html

In [114]:
import os
os.environ['DGLBACKEND'] = 'pytorch'

In [115]:
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

from gradient_descent_the_ultimate_optimizer import gdtuo

import dgl
import dgl.function as fn

import networkx as nx
from matplotlib import pyplot as plt
import numpy as np

import os
import gc

# デバイス設定
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')


In [116]:
def printNPZ(npz):
    for kw in npz.files:
        print(kw, npz[kw])

In [117]:
dirName = '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/'
savedirName = dirName + 'ActiveNet_vp_rotsym_noSelfLoop_batchNorm/'
os.makedirs(savedirName, exist_ok=True)

params = np.load(dirName+'params.npz')
#traj = np.load(dirName+'result.npz')

printNPZ(params)
#printNPZ(traj)

v0 1.0
r 1.0
D 0.1
A 0.0
L 20
rho 1.0
beta 1.0
A_CFs [0.9 0.5]
A_CIL 0.0
cellType_ratio [0.7 0.3]
quiv_colors ['k' 'r']
kappa 0.5
A_Macdonalds [0.5 0.5]
batch_size 400
state_size 3
brownian_size 1
periodic True
t_max 1000
methodSDE heun
isIto False
stepSDE 0.01


In [118]:
if params['periodic']:
    L = torch.tensor(params['L'])
    def calc_dr(r1, r2):
        dr = torch.remainder((r1 - r2), L)
        dr[dr > L/2] = dr[dr > L/2] - L
        return dr
else:
    def calc_dr(r1, r2):
        return r1 - r2
    
def makeGraph(x_data, r_thresh):
        Ndata = x_data.size(0)
        dx = calc_dr(torch.unsqueeze(x_data, 0), torch.unsqueeze(x_data, 1))
        dx = torch.sum(dx**2, dim=2)
        edges = torch.argwhere(torch.logical_and(dx > 0, dx < r_thresh/2))
        return dgl.graph((edges[:,0], edges[:,1]), num_nodes=Ndata)

In [119]:
subdir_list = [f.path for f in os.scandir(dirName) if f.is_dir()]

print(subdir_list)

datadir_list = [f for f in subdir_list if 'result.npz' in [ff.name for ff in os.scandir(f)]]

print(datadir_list)

['/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221026_220625', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221027_020808', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221027_061543', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221027_102254', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221027_144800', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/ActiveNet_vp_rotsym_batchNorm', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221027_190650', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/ActiveNet_vp_rotsym_multiStep_transfer_batchNorm', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/ActiveNet_vp_rotsym_multiStep_fineTuning_batchNorm', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221114_163735', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221114_205206', '/home/uwamichi/jupyter/HiraiwaModel_chem20220922_180005/20221115_004702', '/home/uwamichi/j

In [120]:
dr_thresh = 4
dt = 1
batch_size = 8

N_data = len(datadir_list)

#TR_VA_rate = np.array([0.6, 0.2])

TR_last = 5
VA_last = 7

shuffle_inds = np.arange(N_data, dtype=int)
np.random.shuffle(shuffle_inds)

train_inds = shuffle_inds[:TR_last]
valid_inds = shuffle_inds[TR_last:VA_last]
test_inds = shuffle_inds[VA_last:]

celltype_lst = []

train_x = []
valid_x = []
test_x = []

train_y = []
valid_y = []
test_y = []

train_i_dir = []
valid_i_dir = []
test_i_dir = []

for i_dir, subdirName in enumerate(datadir_list):
    
    traj = np.load(subdirName+'/result.npz')
    
    celltype_lst.append(torch.tensor(traj['celltype_label']).view(-1,1))

    xy_t = torch.tensor(traj['xy'][:-1,:,:])
    v_t = calc_dr(torch.tensor(traj['xy'][1:,:,:]), torch.tensor(traj['xy'][:-1,:,:])) / dt
    p_t = torch.unsqueeze(torch.tensor(traj['theta'][:-1,:]), dim=2)
    w_t = torch.unsqueeze(torch.tensor((traj['theta'][1:,:]-traj['theta'][:-1,:])%(2*np.pi)), dim=2)
    
    if i_dir in train_inds:
        train_x.append(torch.concat((xy_t, p_t), -1))
        train_y.append(torch.concat((v_t, w_t), -1))
        train_i_dir.append(torch.ones([xy_t.size(0)])*i_dir)

    if i_dir in valid_inds:
        valid_x.append(torch.concat((xy_t, p_t), -1))
        valid_y.append(torch.concat((v_t, w_t), -1))
        valid_i_dir.append(torch.ones([xy_t.size(0)])*i_dir)
        
    if i_dir in test_inds:
        test_x.append(torch.concat((xy_t, p_t), -1))
        test_y.append(torch.concat((v_t, w_t), -1))
        test_i_dir.append(torch.ones([xy_t.size(0)])*i_dir)
    
train_dataset = torch.utils.data.TensorDataset(
    torch.concat(train_x, 0), 
    torch.concat(train_y, 0), 
    torch.concat(train_i_dir, 0))

valid_dataset = torch.utils.data.TensorDataset(
    torch.concat(valid_x, 0), 
    torch.concat(valid_y, 0), 
    torch.concat(valid_i_dir, 0))

test_dataset = torch.utils.data.TensorDataset(
    torch.concat(test_x, 0), 
    torch.concat(test_y, 0), 
    torch.concat(test_i_dir, 0))

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, pin_memory=True)
valid_data = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, pin_memory=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, pin_memory=True)

del train_x, train_y, train_i_dir, train_dataset
del valid_x, valid_y, valid_i_dir, valid_dataset
del test_x, test_y, test_i_dir, test_dataset
gc.collect()

#print(data)
#print(data.num_graphs)
#print(data.x)
#print(data.y)
#print(data.edge_index)

396

In [121]:
print(train_inds)

[5 9 3 7 2]


In [122]:
dir(train_data)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_get_shared_seed',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_i

In [123]:
def plotGraph(data):

    # networkxのグラフに変換
    nxg = dgl.to_networkx(data)

    # 可視化のためのページランク計算
    pr = nx.pagerank(nxg)
    pr_max = np.array(list(pr.values())).max()

    # 可視化する際のノード位置
    draw_pos = nx.spring_layout(nxg, seed=0) 

    # ノードの色設定
    cmap = plt.get_cmap('tab10')
    labels = data.y.numpy()
    colors = [cmap(l) for l in labels]

    # 図のサイズ
    fig0 = plt.figure(figsize=(10, 10))

    # 描画
    nx.draw_networkx_nodes(nxg, 
                          draw_pos,
                          node_size=[v / pr_max * 1000 for v in pr.values()])#,
                          #node_color=colors, alpha=0.5)
    nx.draw_networkx_edges(nxg, draw_pos, arrowstyle='-', alpha=0.2)
    nx.draw_networkx_labels(nxg, draw_pos, font_size=10)

    #plt.title('KarateClub')
    plt.show()

    return fig0

In [124]:
class NeuralNet(nn.Module):
    def __init__(self, in_channels, out_channels, Nchannels, dropout=0, batchN=False, flgBias=False):
        super(NeuralNet, self).__init__()

        if dropout:
            self.dropout = nn.Dropout(p=dropout)
        else:
            self.dropout = 0
            
        if batchN:
            self.bNorm1 = nn.BatchNorm1d(Nchannels)
            self.bNorm2 = nn.BatchNorm1d(Nchannels)
            self.bNorm3 = nn.BatchNorm1d(Nchannels)
            
        self.batchN=batchN
        
        self.layer1 = nn.Linear(in_channels, Nchannels, bias=flgBias)
        self.layer2 = nn.Linear(Nchannels, Nchannels, bias=flgBias)
        self.layer3 = nn.Linear(Nchannels, Nchannels, bias=flgBias)
        self.layer4 = nn.Linear(Nchannels, out_channels, bias=flgBias)

        self.activation = nn.ReLU()

    def reset_parameters(self):
        self.layer1.reset_parameters()
        self.layer2.reset_parameters()
        self.layer3.reset_parameters()
        self.layer4.reset_parameters()
        #nn.init.zeros_(self.layer1.weight)
        #nn.init.zeros_(self.layer2.weight)
        #nn.init.zeros_(self.layer3.weight)
        #nn.init.zeros_(self.layer4.weight)
        
    def forward(self, x):
        out = self.activation(self.layer1(x))
        if self.batchN:
            out = self.bNorm1(out)
        if self.dropout:
            out = self.dropout(out)
        
        out = self.activation(self.layer2(out))
        if self.batchN:
            out = self.bNorm2(out)
        if self.dropout:
            out = self.dropout(out)
        
        out = self.activation(self.layer3(out))
        if self.batchN:
            out = self.bNorm3(out)
        if self.dropout:
            out = self.dropout(out)
        
        out = self.layer4(out)

        return out

class ActiveNet(nn.Module):
    def __init__(self, xy_dim, r, dropout=0, batchN=False, bias=False, Nchannels=128):
        super().__init__()

        self.interactNN = NeuralNet(xy_dim*2 + 2, xy_dim, Nchannels, dropout, batchN, bias)

        self.thetaDotNN = NeuralNet(xy_dim*2 + 2, 1, Nchannels, dropout, batchN, bias)
        
        self.selfpropel = nn.Parameter(torch.tensor(0.0, requires_grad=True, device=device))

        #self.Normalizer = nn.Softmax(dim=1)

        self.xy_dim = xy_dim
        
        self.r = r

        self.reset_parameters()

    def reset_parameters(self):
        self.interactNN.reset_parameters()

        self.thetaDotNN.reset_parameters()
        
        nn.init.uniform_(self.selfpropel)

        #self.bias.data.zero_()
        
    def load_celltypes(self, celltype):
        self.celltype = celltype

    def calc_message(self, edges):
        dx = calc_dr(edges.dst['x'], edges.src['x'])

        costheta = torch.cos(edges.dst['theta'])
        sintheta = torch.sin(edges.dst['theta'])

        dx_para = costheta * dx[..., :1] + sintheta * dx[..., 1:]
        dx_perp = costheta * dx[..., 1:] - sintheta * dx[..., :1]

        p_para_src = torch.cos(edges.src['theta'] - edges.dst['theta'])
        p_perp_src = torch.sin(edges.src['theta'] - edges.dst['theta'])

        rot_m_v = self.interactNN(torch.concat((dx_para, dx_perp, 
                                                p_para_src, p_perp_src,
                                                edges.dst['type'], edges.src['type']), -1))

        m_v = torch.concat((costheta * rot_m_v[..., :1] - sintheta * rot_m_v[..., 1:],
                            costheta * rot_m_v[..., 1:] + sintheta * rot_m_v[..., :1]), -1)

        m_theta = self.thetaDotNN(torch.concat((dx_para, dx_perp, 
                                                p_para_src, p_perp_src, 
                                                edges.dst['type'], edges.src['type']), -1))
        
        return {'m': torch.concat((m_v, m_theta), -1)}
        
    def forward(self, xv):
        r_g = makeGraph(xv[..., :self.xy_dim], self.r/2)
        r_g.ndata['x'] = xv[..., :self.xy_dim]
        r_g.ndata['theta'] = xv[..., self.xy_dim:(self.xy_dim+1)]
        r_g.ndata['type'] = self.celltype
        r_g.update_all(self.calc_message, fn.sum('m', 'a'))
        r_g.ndata['a'][..., :self.xy_dim] = r_g.ndata['a'][..., :self.xy_dim] + self.selfpropel * torch.concat((torch.cos(r_g.ndata['theta']), torch.sin(r_g.ndata['theta'])), -1)
        r_g.ndata['a'][..., self.xy_dim:] = r_g.ndata['a'][..., self.xy_dim:]# + r_g.ndata['theta']
        
        return r_g.ndata['a']



In [125]:
def myLoss(out, target):
    dv = torch.sum(torch.square(out[..., :xy_dim] - target[..., :xy_dim]), dim=-1)
    dcos = torch.cos(out[..., xy_dim] - target[..., xy_dim])
    return torch.mean(dv), 1 - torch.mean(dcos)

In [126]:
# モデルのインスタンス生成
xy_dim = 2

model = ActiveNet(xy_dim, dr_thresh, dropout=0, batchN=False, bias=True, Nchannels=128).to(device)
# input data
#data = dataset[0]

# optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)#, weight_decay=5e-4)
#optimizer = torch.optim.Adadelta(model.parameters())#, rho=0.95)#, lr=1e-1, momentum=0.9)

#optim = gdtuo.Adam(alpha=1e-2, beta1=0.9, beta2=0.999, log_eps=-8., optimizer=gdtuo.SGD(1e-3))
#mw = gdtuo.ModuleWrapper(model, optimizer=optim)
#mw.initialize()

scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=5, verbose=True)

val_loss_log = []

val_loss_min = np.Inf

# learnig loop
for epoch in range(300):
    model.train()
    for batch_x, batch_y, batch_i_dir in train_data:
        optimizer.zero_grad()
        lossv = 0
        losstheta = 0
        for ib in range(batch_x.size(0)):
            model.load_celltypes(celltype_lst[int(batch_i_dir[ib])].to(device))
            out = model(batch_x[ib].to(device))
            lv, ltheta = myLoss(out, batch_y[ib].to(device))
            lossv = lossv + lv
            losstheta = losstheta + ltheta
        lossv = lossv / batch_x.size(0)
        losstheta = losstheta / batch_x.size(0)
        (lossv+losstheta).backward()
        optimizer.step()
    model.eval()
    val_lossv = 0
    val_losstheta = 0
    val_count = 0
    with torch.no_grad():
        for batch_x, batch_y, batch_i_dir in valid_data:
            for ib in range(batch_x.size(0)):
                model.load_celltypes(celltype_lst[int(batch_i_dir[ib])].to(device))
                val_out = model(batch_x[ib].to(device))
                lv, ltheta = myLoss(val_out, batch_y[ib].to(device))
                val_lossv = val_lossv + lv
                val_losstheta = val_losstheta + ltheta
            val_count = val_count + batch_x.size(0)
    val_lossv = val_lossv/val_count
    val_losstheta = val_losstheta/val_count
    val_loss = val_lossv + val_losstheta
    scheduler.step(val_loss)
    print('Epoch %d | train Loss: [%.4f, %.4f] | valid Loss: [%.4f, %.4f]' % (epoch,
                                                                              lossv.item(), 
                                                                              losstheta.item(),
                                                                              val_lossv.item(), 
                                                                              val_losstheta.item()))
    val_loss_log.append([val_lossv.cpu().item(), val_losstheta.cpu().item()])
    if val_loss.item() < val_loss_min:
        stored_model = model
        val_loss_min = val_loss.item()

Epoch 0 | train Loss: [0.0975, 0.0471] | valid Loss: [0.1033, 0.0567]
Epoch 1 | train Loss: [0.0911, 0.0460] | valid Loss: [0.0965, 0.0548]
Epoch 2 | train Loss: [0.0883, 0.0458] | valid Loss: [0.0936, 0.0543]
Epoch 3 | train Loss: [0.0865, 0.0458] | valid Loss: [0.0916, 0.0540]
Epoch 4 | train Loss: [0.0853, 0.0457] | valid Loss: [0.0900, 0.0539]
Epoch 5 | train Loss: [0.0841, 0.0457] | valid Loss: [0.0884, 0.0538]
Epoch 6 | train Loss: [0.0829, 0.0456] | valid Loss: [0.0870, 0.0537]
Epoch 7 | train Loss: [0.0818, 0.0456] | valid Loss: [0.0856, 0.0537]
Epoch 8 | train Loss: [0.0807, 0.0456] | valid Loss: [0.0842, 0.0536]
Epoch 9 | train Loss: [0.0795, 0.0455] | valid Loss: [0.0828, 0.0536]
Epoch 10 | train Loss: [0.0784, 0.0455] | valid Loss: [0.0814, 0.0535]
Epoch 11 | train Loss: [0.0771, 0.0455] | valid Loss: [0.0798, 0.0535]
Epoch 12 | train Loss: [0.0757, 0.0454] | valid Loss: [0.0782, 0.0535]
Epoch 13 | train Loss: [0.0740, 0.0454] | valid Loss: [0.0766, 0.0534]
Epoch 14 | train

Epoch 116 | train Loss: [0.0290, 0.0437] | valid Loss: [0.0385, 0.0515]
Epoch 117 | train Loss: [0.0290, 0.0437] | valid Loss: [0.0385, 0.0515]
Epoch 118 | train Loss: [0.0290, 0.0437] | valid Loss: [0.0385, 0.0515]
Epoch 119 | train Loss: [0.0289, 0.0436] | valid Loss: [0.0385, 0.0514]
Epoch 120 | train Loss: [0.0289, 0.0436] | valid Loss: [0.0385, 0.0514]
Epoch 121 | train Loss: [0.0289, 0.0436] | valid Loss: [0.0384, 0.0514]
Epoch 122 | train Loss: [0.0289, 0.0436] | valid Loss: [0.0384, 0.0514]
Epoch 123 | train Loss: [0.0289, 0.0436] | valid Loss: [0.0384, 0.0514]
Epoch 124 | train Loss: [0.0289, 0.0435] | valid Loss: [0.0384, 0.0513]
Epoch 125 | train Loss: [0.0289, 0.0435] | valid Loss: [0.0384, 0.0513]
Epoch 126 | train Loss: [0.0289, 0.0435] | valid Loss: [0.0384, 0.0513]
Epoch 127 | train Loss: [0.0289, 0.0435] | valid Loss: [0.0384, 0.0513]
Epoch 128 | train Loss: [0.0289, 0.0435] | valid Loss: [0.0383, 0.0513]
Epoch 129 | train Loss: [0.0289, 0.0434] | valid Loss: [0.0383, 

Epoch 230 | train Loss: [0.0285, 0.0425] | valid Loss: [0.0375, 0.0496]
Epoch 231 | train Loss: [0.0285, 0.0425] | valid Loss: [0.0375, 0.0496]
Epoch 232 | train Loss: [0.0285, 0.0425] | valid Loss: [0.0375, 0.0496]
Epoch 233 | train Loss: [0.0285, 0.0425] | valid Loss: [0.0375, 0.0496]
Epoch 234 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 235 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 236 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 237 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 238 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 239 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 240 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 241 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 242 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 0.0496]
Epoch 243 | train Loss: [0.0285, 0.0424] | valid Loss: [0.0375, 

In [127]:
model.state_dict()

OrderedDict([('selfpropel', tensor(0.9251, device='cuda:1')),
             ('interactNN.layer1.weight',
              tensor([[ 0.2185,  0.2012,  0.0633, -0.1603, -0.2546,  0.2063],
                      [ 0.0789,  0.0199, -0.2185, -0.0179,  0.0370,  0.2715],
                      [-0.0823,  0.1029, -0.3984, -0.0192,  0.3186, -0.0922],
                      [ 0.0451,  0.2858, -0.0804, -0.3068,  0.0925,  0.0947],
                      [-0.4581,  0.0992, -0.2529, -0.3365,  0.3901, -0.0395],
                      [-0.0803,  0.0635, -0.0571,  0.2846, -0.2998,  0.3691],
                      [ 0.4822,  0.2574,  0.0750,  0.1314,  0.0157, -0.3035],
                      [ 0.4956, -0.4538, -0.0259, -0.0735, -0.0517, -0.0602],
                      [-0.5238, -0.5016, -0.0672, -0.1873, -0.1848, -0.1180],
                      [ 0.3227, -0.3686, -0.2994,  0.1840, -0.3364,  0.3206],
                      [-0.3876, -0.3320, -0.1246, -0.0276, -0.3060, -0.2636],
                      [ 0.0414, -0.223

In [128]:
# モデルを評価モードに設定
stored_model.eval()

# 推論
test_lossv = 0
test_losstheta = 0
test_count = 0
with torch.no_grad():
    for batch_x, batch_y, batch_i_dir in test_data:
        for ib in range(batch_x.size(0)):
            model.load_celltypes(celltype_lst[int(batch_i_dir[ib])].to(device))
            test_out = model(batch_x[ib].to(device))
            lv, ltheta = myLoss(test_out, batch_y[ib].to(device))
            test_lossv = test_lossv + lv
            test_losstheta = test_losstheta + ltheta
        test_count = test_count + batch_x.size(0)
test_lossv = test_lossv/test_count
test_losstheta = test_losstheta/test_count
print('test Loss: [%.4f, %.4f]' % (test_lossv.item(), test_losstheta.item()))
test_loss = [test_lossv.item(), test_losstheta.item()]

test Loss: [0.0373, 0.0494]


In [129]:
import pickle
import datetime

now = datetime.datetime.now()
nowstr = now.strftime('%Y%m%d_%H%M%S')

os.makedirs(savedirName + nowstr + '/', exist_ok=True)

In [130]:
stored_model.selfpropel.detach()

tensor(0.9251, device='cuda:1')

In [131]:
stored_model = stored_model.to('cpu')

filename1 = savedirName + nowstr + '/' + nowstr + '_Model.pkl'
with open(filename1, "wb") as f:
    pickle.dump(stored_model, f)

filename1_2 = savedirName + nowstr + '/' + nowstr + '_Model.pt'
torch.save(stored_model, filename1_2)

filename2 = savedirName + nowstr + '/' + nowstr
torch.save(stored_model.interactNN.state_dict(), filename2 + '_interactNN.pkl')
torch.save(stored_model.thetaDotNN.state_dict(), filename2 + '_thetaDotNN.pkl')
torch.save(stored_model.selfpropel.detach(), filename2 + '_selfpropel.pkl')

filename3 = savedirName + nowstr + '/' + nowstr + '_Separation.npz'
np.savez(filename3, dr_thresh=dr_thresh, batch_size=batch_size,
         train_inds=train_inds, valid_inds=valid_inds, test_inds=test_inds, 
         val_loss_log=val_loss_log, test_loss=test_loss)

filename4 = savedirName + nowstr + '/' + nowstr + '_optimizer.pt'
torch.save(optimizer, filename4)